In [2]:
pip install scikeras

  Using cached scikeras-0.12.0-py3-none-any.whl.metadata (4.0 kB)
Using cached scikeras-0.12.0-py3-none-any.whl (27 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   --- ------------------------------------ 0.1/1.5 MB 3.3 MB/s eta 0:00:01
   ------- -------------------------------- 0.3/1.5 MB 4.3 MB/s eta 0:00:01
   ------------------- -------------------- 0.7/1.5 MB 5.7 MB/s eta 0:00:01
   ---------------------------------------  1.5/1.5 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 7.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.10.0 requires libclang>=13.0.0, which is not installed.
tensorflow 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.


In [12]:
# Imports here
import numpy as np
import pandas as pd
import json
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from scikeras.wrappers import KerasClassifier, KerasRegressor
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

In [6]:
# Dataset here
file = open("../dataset/processed_reviews.json", 'r', encoding='utf8')
dataset_dict = json.load(file)
df = pd.DataFrame(dataset_dict)
df.drop(columns=['date_review', 'firm','job_title', 'current', 'headline', 'pros', 'cons', 'duration'], inplace=True)

In [ ]:
# # 1hot "x" "o" and "r"
# xor_dict = {"x": [1, 0, 0, 0], "o": [0, 1, 0, 0], "r": [0, 0, 1, 0], "v": [0, 0, 0, 1]}

# for d in dataset_dict:
#     # leave years alone, 1-hot months
#     year, month = d.pop('date_review').split("-")[:2]
#     d['year_review'] = int(year)
#     d['one_hot_months'] = [0] * 12
#     d['one_hot_months'][int(month) - 1] = 1
    
#     d['recommend'] = xor_dict[d['recommend']]
#     d['ceo_approv'] = xor_dict[d['ceo_approv']]
#     d['outlook'] = xor_dict[d['outlook']]

In [7]:
# Min-max normalization
scaler = MinMaxScaler()
numeric_cols = df.select_dtypes(include=['int', 'float']).columns
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

#One hot encode
one_hot_encoded = pd.get_dummies(df[['recommend', 'ceo_approv', 'outlook']])
df = pd.concat([df, one_hot_encoded], axis=1)
df.drop(columns=['recommend', 'ceo_approv', 'outlook'], inplace=True)

df.head()

,overall_rating,work_life_balance,culture_values,career_opp,comp_benefits,senior_mgmt,recommend_o,recommend_v,recommend_x,ceo_approv_o,ceo_approv_r,ceo_approv_v,ceo_approv_x,outlook_o,outlook_r,outlook_v,outlook_x
0,0.25,0.50,0.00,0.25,0.0,0.75,False,False,True,True,False,False,False,False,True,False,False
1,0.00,0.00,0.00,0.00,0.0,0.00,False,False,True,True,False,False,False,False,False,False,True
2,0.00,0.25,0.00,0.25,0.0,0.00,False,False,True,True,False,False,False,False,False,False,True
3,0.50,0.75,0.25,0.25,0.5,0.25,True,False,False,False,True,False,False,False,True,False,False
4,0.00,0.00,0.00,0.00,0.0,0.00,False,False,True,True,False,False,False,False,True,False,False


In [8]:
#Spliting the data
X = df.drop(columns=['overall_rating'])
y = df['overall_rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [14]:
def buildSigmoidPerceptron():
    model = Sequential([
        Dense(1, activation = 'sigmoid', input_dim = X.shape[1]),
    ])
    optimizer = SGD(learning_rate=0.3)
    model.compile(optimizer=optimizer, loss='mse', metrics=['MSE'])
    return(model)

estimator = KerasRegressor(model=buildSigmoidPerceptron, epochs=10, batch_size=1000, verbose=0)
estimator.fit(X_train, y_train)
y_pred = estimator.predict(X_test)
pd.DataFrame(y_pred)

,0
0,0.207016
1,0.508043
2,0.713205
3,0.938203
4,0.255673
...,...
75735,0.675715
75736,0.910485
75737,0.909996
75738,0.929825


In [25]:
# Surprisingly low MSE with just perceptron! Did I overlook something?
MSE = sum((y_pred - y_test)**2)/y_test.size
print(MSE)

0.03129764562130118
